<a href="https://colab.research.google.com/github/NaveenOburi/Track-And-Consult-Healthcare-services/blob/master/Assignment__1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


import the libraries to work with the dataset

In [0]:
# Import the pandas library to read our dataset
import pandas as pd
#Import the numpy library to work with and manipulate the data
import numpy as np
# Import the pytorch library
import torch
# Import the time
import time
# to show the plot 
import matplotlib.pyplot as plt
# Get the train/test split package from sklearn for preparing our dataset to train and test the model with
from sklearn.model_selection import train_test_split
# Import the 1D convolution layer
# Since we’re inputting a 1 dimensional row of data, we can’t use 2D or 3D
from torch.nn import Conv1d
#Import the max pooling layer
from torch.nn import MaxPool1d
# Import the flatten layer
from torch.nn import Flatten
# Import the linear layer
from torch.nn import Linear
# Import the Sigmoid activation function
from torch.nn.functional import sigmoid
# Import the DataLoader and TensorDataset libraries from PyTorch
# to work with the datasets
from torch.utils.data import DataLoader,TensorDataset


To read , clean the dataset alos print the first ten records of the housing dataset.

In [0]:
dataset=pd.read_csv('/content/housing.csv')
dataset=dataset.dropna()
# Print the first ten records of the dataset.
dataset.head(10)


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
5,-122.25,37.85,52.0,919.0,213.0,413.0,193.0,4.0368,269700.0,NEAR BAY
6,-122.25,37.84,52.0,2535.0,489.0,1094.0,514.0,3.6591,299200.0,NEAR BAY
7,-122.25,37.84,52.0,3104.0,687.0,1157.0,647.0,3.1200,241400.0,NEAR BAY
8,-122.26,37.84,42.0,2555.0,665.0,1206.0,595.0,2.0804,226700.0,NEAR BAY
9,-122.25,37.84,52.0,3549.0,707.0,1551.0,714.0,3.6912,261100.0,NEAR BAY


Train/test split ratio of 70:30 using sklearn.model selection library with random state set to 2003.

In [0]:
# predict the median_house_value column
y=dataset['median_house_value']

#The remainder of the columns will be used to predict Y
#Select from the " column to the median_income column
x=dataset.loc[:,'longitude':'median_income']

#Test size 0.3 means test sample will be 30% and training will be 70%. 
#Thus, ratio of train:test is 70:30
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3,random_state=2003)

# Converts the datasets to numpy arrays to work with our PyTorch model
x_train_np=x_train.to_numpy()
y_train_np=y_train.to_numpy()

# Convert the testing data
x_test_np=x_test.to_numpy()
y_test_np=y_test.to_numpy()

#Size of Trainable parameters can be find by checking the shape of training data.
x_train_np.shape

(14303, 8)

We use here different activation the Sigmoid Function curve looks like an S-shape.


creating the class MUST be a subclass of torch.nn.Module . the networks 

In [0]:
# the class MUST be a subclass of torch.nn.Module
class cnnRegressor(torch.nn.Module):
  # Define the initialization method
  def __init__(self,batch_size,inputs,outputs):

# Initialize the superclass and store the parameters
    super(cnnRegressor, self).__init__()
    self.batch_size=batch_size
    self.inputs=inputs
    self.outputs=outputs
    # Define the input layer (input channels, output channels, kernel size)
    self.input_layer=Conv1d(inputs, batch_size, 1)
    # Define a max pooling layer (kernel size)
    self.max_pooling_layer=MaxPool1d(1)
    # Define another convolution layer
    self.conv_layer=Conv1d(batch_size,128,1)
    # Define a flatten layer
    self.flatten_layer=Flatten()
    # Define a linear layer ( outputs)
    self.linear_layer=Linear(128,64)
    # Finally, define the output layer
    self.output_layer=Linear(64,outputs)
    # Define a method to feed inputs through the model
  def feed(self, input):
    # Reshape the entry so it can be fed to the input layer
    # Although we’re using 1D convolution, it still expects a 3D array to process in a 1D fashion
     input = input.reshape(self.batch_size, self.inputs, 1)

  # Get the output of the first layer and run it through the the Sigmoid activation function.
     output = sigmoid(self.input_layer(input))

     # Get the output of the max pooling layer
     output = sigmoid(self.max_pooling_layer(output))
                                      
   # Get the output of the second convolution layer and run it through the Sigmoid activation function
     output = sigmoid(self.conv_layer(output))

     # Get the output of the flatten layer
     output = self.flatten_layer(output)
     # Get the output of the linear layer and run it through the Sigmoid activation function
     output = self.linear_layer(output)
     # Finally, get the output of the output layer and return it
     output = self.output_layer(output)
     return output

In [0]:
# Import the Adam (stochastic gradient descent) package from pytorch for
# our optimizer
from torch.optim import Adam
# Import the L1Loss (mean absolute error loss) package from pytorch for
# our performance measure
from torch.nn import L1Loss
# Import the R^2 score package from pytorch's ignite for our score measure
# This package is not installed by default so the next line does that
!pip install pytorch-ignite
from ignite.contrib.metrics.regression.r2_score import R2Score


The kernel_size is 1X1. Due to this convolution operation is also known as 1-d convolution.  Define the  CNN model




In [0]:
# Define the batch size we'd like to use
batch_size = 64
#(batch size, X columns, Y columns)
model =cnnRegressor(batch_size, x.shape[1], 1)
# Set the model to use the GPU for processing
model.cuda()

# saving the model
torch.save(model.state_dict(), "student_id:(0873122)1dconv reg.model")


 Number of trainable parameters.

In [0]:
print(sum(p.numel() for p in model.parameters()))
#pytorch_total_params = sum(p.numel() for p in model.parameters())


Create a method for running the batches of data through the model.

In [0]:
# This method will return the average L1 loss and R^2 score of the passed model on the passed DataLoader
def model_loss(model, dataset, train = False, optimizer = None):

# Cycle through the batches and get the average L1 loss
  performance = L1Loss()
  score_metric = R2Score()

  avg_loss = 0

  avg_score = 0

  count = 0

  for input, output in iter(dataset):

# Get the model's predictions for the training dataset
    predictions = model.feed(input)

# Get the model's loss
    loss = performance(predictions, output)

# Get the model's R^2 score
    score_metric.update([predictions, output])
    score = score_metric.compute()

    if(train):

# Clear any errors so they don't cummulate
      optimizer.zero_grad()

# Compute the gradients for our optimizer
      loss.backward()

# Use the optimizer to update the model's parameters based on the gradients
      optimizer.step()

# Store the loss and update the counter
    avg_loss += loss.item()
    avg_score += score
    count += 1

  return avg_loss / count, avg_score/ count

Here we increase the size of the epochs and change the optimizer and the Learning rate.

In [0]:
# Define the number of epochs to train for 

epochs = 100

# Define the performance measure and optimizer Import the Adam (stochastic gradient descent) package from pytorch for
# our optimizer
optimizer = Adam(model.parameters(), lr = 1e-3)

# Convert the training set into torch variables for our model using the GPU
# as floats. The reshape is to remove a warning pytorch outputs otherwise.
inputs = torch.from_numpy(x_train_np).cuda().float()
outputs = torch.from_numpy(y_train_np.reshape(y_train_np.shape[0], 1)).cuda().float()

# Create a DataLoader instance to work with our batches
tensor = TensorDataset(inputs, outputs)
loader = DataLoader(tensor, batch_size, shuffle=True, drop_last=True)

start_time = time.time()
# Start the training loop
for epoch in range(epochs):
  # Cycle through the batches and get the average loss
  avg_loss, avg_r2_score = model_loss (model, loader, train = True, optimizer = optimizer)

# Output the average loss
print("Epoch " + str(epoch + 1) + ":\n\tLoss = " + str(avg_loss) + "\n\tR^2 Score = " + str(avg_r2_score))

 #end_time= time.time()

model = model.eval()
print('training time elapsed: %.2f min' % ((time.time() - start_time)/60))
print('training time elapsed: %.2f min' % ((time.time() - start_time)/60))

Finally, we will test the model to see how it performs on the testing dataset

In [0]:
# Convert the testing set into torch variables for our model using the GPU as floats
inputs = torch.from_numpy(x_test_np).cuda().float()
outputs = torch.from_numpy(y_test_np.reshape(y_test_np.shape[0], 1)).cuda().float()

# Create a DataLoader instance to work with our batches
tensor = TensorDataset(inputs, outputs)
loader = DataLoader(tensor, batch_size, shuffle = True, drop_last=True)

# Output the average performance of the model
avg_loss, avg_r2_score = model_loss(model, loader)
print("The model's L1 loss is: " + str(avg_loss))
print("The model's R^2 score is: " + str(avg_r2_score))

Figure 1: Example sub-plots of first eighteen samples from the dataset in a single figure

In [0]:
columns = dataset.columns.drop(['longitude','latitude','housing_median_age','total_rooms','total_bedrooms','population','households','median_income'])

x= range(0, dataset.shape[0])
fig, ax = plt.subplots()

for column in dataset:
  ax.plot(x, dataset[column],label=column)

dataset.plot(subplots=True)

ax.set_title('housing Dataset')



In [0]:
start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)

start.record()
z = x + y
end.record()

# Waits for everything to finish running
torch.cuda.synchronize()

print(start.elapsed_time(end))